In [1]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials

def import_df_sheets(spreadsheet_url, worksheet_name):
    """
    Conecta a Google Sheets usando gspread y convierte una hoja específica a un DataFrame.
    
    Parámetros:
        spreadsheet_url (str): URL de la hoja de cálculo de Google Sheets.
        worksheet_name (str): Nombre de la hoja que deseas importar.
        expected_headers (list, opcional): Lista de cabeceras esperadas para los datos.
    
    Retorna:
        pd.DataFrame: Contenido de la hoja en un DataFrame.
    """
    try:
        # Definir el alcance
        scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
        
        # Cargar credenciales
        creds = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
        client = gspread.authorize(creds)
        
        # Abrir la hoja de cálculo
        spreadsheet = client.open_by_url(spreadsheet_url)
        
        # Seleccionar la hoja específica
        worksheet = spreadsheet.worksheet(worksheet_name)
        
        data = worksheet.get_all_values()
        
        # Convertir a DataFrame
        df = pd.DataFrame(data)
        
        return df
    
    except Exception as e:
        print(f"Error al importar la hoja: {e}")
        return None

# Uso de la función




In [4]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(df_raw,column=2,ini=3,columna='Categoria'):
    # Remove rows 0 to 1, set row 2 as header, drop rows where column 'Categoria' is empty
    df_raw.columns = df_raw.iloc[column]  # Set row 2 as header
    df_raw = df_raw[ini:]  # Remove rows 0 to 2
    # Filter rows where 'Categoria' is not empty or null
    df_raw = df_raw[df_raw[columna].notna() & (df_raw[columna] != "")]
    # Reset index and start from 1
    df_raw.reset_index(drop=True)
    df_raw.index += 1
    return df_raw

def convert_column_to_numeric(df, column_name):
    """Converts a specified column in a DataFrame to numeric, handling '.' and ',' as decimal separators.

    Args:
        df: The input DataFrame.
        column_name: The name of the column to convert.

    Returns:
        The DataFrame with the specified column converted to numeric.
    """
    df[column_name] = df[column_name].astype(str).str.replace('.', '', regex=False)
    df[column_name] = df[column_name].astype(str).str.replace(',', '.', regex=False)
    df[column_name] = pd.to_numeric(df[column_name], errors='coerce')
    return df
def safe_filter_columns(df, columnas):
    """
    Filtra un DataFrame por una lista de columnas de forma segura, 
    ignorando las columnas que no existan.
    """
    columnas_existentes = [col for col in columnas if col in df.columns]
    return df[columnas_existentes]

In [5]:
sheet_url = "https://docs.google.com/spreadsheets/d/1jujbeFMzGuSpcK9blAXYZ8La9Aa8Y3-3xiyuDl-3tAE/edit#gid=1962069249"
df_recurrentes  = clean_data(import_df_sheets(sheet_url, "Recurrentes 24-26"))
df_no_recurrentes  = clean_data(import_df_sheets(sheet_url, "No Recurrentes 24-26"))
df_adicionales  = clean_data(import_df_sheets(sheet_url, "RepuestosAdicionales"))
df_lubricantes  = clean_data(import_df_sheets(sheet_url, "Lubricantes"),column=1,ini=2)
# Concatenar todos los dataframes

# Seleccion de columnas que se van a usar
#df_recurrentes.columns
columnas = ['Categoria', 'Proceso', 'Ubicación', 'Tipo de trabajo', 'Tipo de Bien',
       'TDB/LOCAL', 'Partida', 'ID TDB o ID Almacén', 'ID Cotizado',
       'Descripción del Bien', 'Unid.', 'Q LMO/24', 'Q LPL/24', 'Q LAM/24',
       'Q LR/24', 'Q LN/24', 'Q LAZ/24', 'Q LB/24', 'Q LCE/24', 'Q LCA/24',
       'Q LV/24', 'Q TRM/24', 'Q LMO/25', 'Q LPL/25', 'Q LAM/25',
       'Q LR/25', 'Q LN/25', 'Q LAZ/25', 'Q LB/25', 'Q LCE/25', 'Q LCA/25',
       'Q LV/25', 'Q TRM/25', 'Q LMO/26', 'Q LPL/26', 'Q LAM/26',
       'Q LR/26', 'Q LN/26', 'Q LAZ/26', 'Q LB/26', 'Q LCE/26', 'Q LCA/26',
       'Q LV/26', 'Q TRM/26', 'P.U. $us.', 'P.U. Bs',
       #,'Q 2024', 'Q 2025','Q 2026',  'Q Total'
       'Observaciones','Comentarios']

df_recurrentes  =  safe_filter_columns(df_recurrentes, columnas) 
df_no_recurrentes  = safe_filter_columns(df_no_recurrentes, columnas) 
df_adicionales  = safe_filter_columns(df_adicionales, columnas) 
df_lubricantes  = safe_filter_columns(df_lubricantes, columnas)

In [6]:
df_recurrentes

2,Categoria,Proceso,Ubicación,Tipo de trabajo,Tipo de Bien,TDB/LOCAL,Partida,ID TDB o ID Almacén,ID Cotizado,Descripción del Bien,...,Q LAZ/26,Q LB/26,Q LCE/26,Q LCA/26,Q LV/26,Q TRM/26,P.U. $us.,P.U. Bs,Observaciones,Comentarios
4,Repuestos Electromecánicos recurrentes,2,Torres,Reacondionamiento y cambio de Poleas de torre,Repuesto Mecánico,TDB,39800,2-01-00011,10431740,GUARN.DE GOMA 501 485X100/R40,...,150,41,66,20,50,0,"401,33","2.793,26",,
5,Repuestos Electromecánicos recurrentes,2,Torres,Reacondionamiento y cambio de Poleas de torre,Repuesto Mecánico,TDB,39800,2-01-00012,10434107,GUARN.DE GOMA 420 422X 93/R34,...,0,63,76,10,65,0,"317,23","2.207,92",,
6,Repuestos Electromecánicos recurrentes,,Torres,Reacondionamiento y cambio de Poleas de torre,Rodamientos,LOCAL,39800,2-01-00077,10501989,RRB DIN 625/1- 6310-2RS1-C3GLE VT210 (90% +/-5...,...,300,120,132,50,100,0,"62,07","432,01",Se descontaron cantidades adquiridas 2024,
7,Repuestos Electromecánicos recurrentes,2,Estaciones,Reacondicionamiento y Cambio de poleas de esta...,Repuesto Mecánico,TDB,39800,2-01-00011,10431740,GUARN.DE GOMA 501 485X100/R40,...,120,60,60,15,0,0,"401,33","2.793,26",,
8,Repuestos Electromecánicos recurrentes,2,Estaciones,Reacondicionamiento y Cambio de poleas de esta...,Repuesto Mecánico,TDB,39800,2-01-00074,10461436,GUARN.DE GOMA 403 403X 93/R38,...,144,36,36,15,0,0,"275,00","1.914,00",,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,Repuestos Electromecánicos recurrentes,2,Vehículo,Manto. Pinza,Repuesto Mecánico,TDB,39800,2-01-00349,11125072,TUERCA HEX.ISO4035- 8 M18 GALVANIZADA ...,...,0,10,10,5,0,0,"0,39","2,71",,
138,Repuestos Electromecánicos recurrentes,2,Vehículo,10% Suspensiones de Cabinas,Repuesto Mecánico,TDB,39800,2-01-00286,10000079,TUERCA HEXAG. DIN 985-8 M10 ELECTROGALV. (C.EL...,...,21,13,16,3,0,0,"0,23","1,60",,
139,Repuestos Electromecánicos recurrentes,2,Torres,Reacondicionamiento de Poleas Palpadoras,Repuesto Mecánico,TDB,39800,2-01-00835,10006287,"RES.C. A=1,00/DIA.7,0/LSC=20,0 EE= 7,5; RES.D...",...,15,0,0,0,8,0,"0,27","1,88",,
140,Repuestos Electromecánicos recurrentes,2,Torres,Reacondicionamiento de Poleas Palpadoras,Repuesto Mecánico,TDB,39800,2-01-00840,10873673,"ARANDELA ISO 7089 M20 300HV, FLZNNC-720H-L, RE...",...,0,2,2,1,2,0,"0,65","4,52",,


In [127]:
columnas_a_derretir = ['Q LMO/24', 'Q LPL/24', 'Q LAM/24',
       'Q LR/24', 'Q LN/24', 'Q LAZ/24', 'Q LB/24', 'Q LCE/24', 'Q LCA/24',
       'Q LV/24', 'Q TRM/24', 'Q LMO/25', 'Q LPL/25', 'Q LAM/25',
       'Q LR/25', 'Q LN/25', 'Q LAZ/25', 'Q LB/25', 'Q LCE/25', 'Q LCA/25',
       'Q LV/25', 'Q TRM/25', 'Q LMO/26', 'Q LPL/26', 'Q LAM/26',
       'Q LR/26', 'Q LN/26', 'Q LAZ/26', 'Q LB/26', 'Q LCE/26', 'Q LCA/26',
       'Q LV/26', 'Q TRM/26'
       #'Q 2024','Q 2025','Q 2026', 'Q Total'
       ]

df_melted = (
    df_recurrentes
    .melt(
        id_vars=[col for col in df_recurrentes.columns if col not in columnas_a_derretir],
        value_vars=columnas_a_derretir,
        var_name='linea',
        value_name='value'
    )
)
df_melted


,Categoria,Proceso,Ubicación,Tipo de trabajo,Tipo de Bien,TDB/LOCAL,Partida,ID TDB o ID Almacén,ID Cotizado,Descripción del Bien,...,Q 2024,Q 2025,Q 2026,Q Total,P.U. $us.,P.U. Bs,Observaciones,Comentarios,linea,value
0,Repuestos Electromecánicos recurrentes,2,Torres,Reacondionamiento y cambio de Poleas de torre,Repuesto Mecánico,TDB,39800,2-01-00011,10431740,GUARN.DE GOMA 501 485X100/R40,...,"707,00","877,00","877,00","2.461,00","401,33","2.793,26",,,Q LMO/24,150
1,Repuestos Electromecánicos recurrentes,2,Torres,Reacondionamiento y cambio de Poleas de torre,Repuesto Mecánico,TDB,39800,2-01-00012,10434107,GUARN.DE GOMA 420 422X 93/R34,...,"174,00","579,00","579,00","1.332,00","317,23","2.207,92",,,Q LMO/24,37
2,Repuestos Electromecánicos recurrentes,,Torres,Reacondionamiento y cambio de Poleas de torre,Rodamientos,LOCAL,39800,2-01-00077,10501989,RRB DIN 625/1- 6310-2RS1-C3GLE VT210 (90% +/-5...,...,"1.099,00","1.942,00","1.942,00","4.983,00","62,07","432,01",Se descontaron cantidades adquiridas 2024,,Q LMO/24,206
3,Repuestos Electromecánicos recurrentes,2,Estaciones,Reacondicionamiento y Cambio de poleas de esta...,Repuesto Mecánico,TDB,39800,2-01-00011,10431740,GUARN.DE GOMA 501 485X100/R40,...,"67,00","471,00","471,00","1.009,00","401,33","2.793,26",,,Q LMO/24,7
4,Repuestos Electromecánicos recurrentes,2,Estaciones,Reacondicionamiento y Cambio de poleas de esta...,Repuesto Mecánico,TDB,39800,2-01-00074,10461436,GUARN.DE GOMA 403 403X 93/R38,...,"168,00","495,00","495,00","1.158,00","275,00","1.914,00",,,Q LMO/24,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4054,Repuestos Electromecánicos recurrentes,2,Vehículo,Manto. Pinza,Repuesto Mecánico,TDB,39800,2-01-00349,11125072,TUERCA HEX.ISO4035- 8 M18 GALVANIZADA ...,...,"60,00","99,00","99,00","258,00","0,39","2,71",,,Q TRM/26,0
4055,Repuestos Electromecánicos recurrentes,2,Vehículo,10% Suspensiones de Cabinas,Repuesto Mecánico,TDB,39800,2-01-00286,10000079,TUERCA HEXAG. DIN 985-8 M10 ELECTROGALV. (C.EL...,...,"0,00","130,00","130,00","260,00","0,23","1,60",,,Q TRM/26,0
4056,Repuestos Electromecánicos recurrentes,2,Torres,Reacondicionamiento de Poleas Palpadoras,Repuesto Mecánico,TDB,39800,2-01-00835,10006287,"RES.C. A=1,00/DIA.7,0/LSC=20,0 EE= 7,5; RES.D...",...,"72,00","72,00","72,00","216,00","0,27","1,88",,,Q TRM/26,0
4057,Repuestos Electromecánicos recurrentes,2,Torres,Reacondicionamiento de Poleas Palpadoras,Repuesto Mecánico,TDB,39800,2-01-00840,10873673,"ARANDELA ISO 7089 M20 300HV, FLZNNC-720H-L, RE...",...,"12,00","12,00","12,00","36,00","0,65","4,52",,,Q TRM/26,0


In [129]:
df_melted = convert_column_to_numeric(df_melted, 'value')
df_melted.dtypes

Categoria                object
Proceso                  object
Ubicación                object
Tipo de trabajo          object
Tipo de Bien             object
TDB/LOCAL                object
Partida                  object
ID TDB o ID Almacén      object
ID Cotizado              object
Descripción del Bien     object
Unid.                    object
Q 2024                   object
Q 2025                   object
Q 2026                   object
Q Total                  object
P.U. $us.                object
P.U. Bs                  object
Observaciones            object
Comentarios              object
linea                    object
value                   float64
dtype: object

In [130]:
df_melted

,Categoria,Proceso,Ubicación,Tipo de trabajo,Tipo de Bien,TDB/LOCAL,Partida,ID TDB o ID Almacén,ID Cotizado,Descripción del Bien,...,Q 2024,Q 2025,Q 2026,Q Total,P.U. $us.,P.U. Bs,Observaciones,Comentarios,linea,value
0,Repuestos Electromecánicos recurrentes,2,Torres,Reacondionamiento y cambio de Poleas de torre,Repuesto Mecánico,TDB,39800,2-01-00011,10431740,GUARN.DE GOMA 501 485X100/R40,...,"707,00","877,00","877,00","2.461,00","401,33","2.793,26",,,Q LMO/24,150.0
1,Repuestos Electromecánicos recurrentes,2,Torres,Reacondionamiento y cambio de Poleas de torre,Repuesto Mecánico,TDB,39800,2-01-00012,10434107,GUARN.DE GOMA 420 422X 93/R34,...,"174,00","579,00","579,00","1.332,00","317,23","2.207,92",,,Q LMO/24,37.0
2,Repuestos Electromecánicos recurrentes,,Torres,Reacondionamiento y cambio de Poleas de torre,Rodamientos,LOCAL,39800,2-01-00077,10501989,RRB DIN 625/1- 6310-2RS1-C3GLE VT210 (90% +/-5...,...,"1.099,00","1.942,00","1.942,00","4.983,00","62,07","432,01",Se descontaron cantidades adquiridas 2024,,Q LMO/24,206.0
3,Repuestos Electromecánicos recurrentes,2,Estaciones,Reacondicionamiento y Cambio de poleas de esta...,Repuesto Mecánico,TDB,39800,2-01-00011,10431740,GUARN.DE GOMA 501 485X100/R40,...,"67,00","471,00","471,00","1.009,00","401,33","2.793,26",,,Q LMO/24,7.0
4,Repuestos Electromecánicos recurrentes,2,Estaciones,Reacondicionamiento y Cambio de poleas de esta...,Repuesto Mecánico,TDB,39800,2-01-00074,10461436,GUARN.DE GOMA 403 403X 93/R38,...,"168,00","495,00","495,00","1.158,00","275,00","1.914,00",,,Q LMO/24,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4054,Repuestos Electromecánicos recurrentes,2,Vehículo,Manto. Pinza,Repuesto Mecánico,TDB,39800,2-01-00349,11125072,TUERCA HEX.ISO4035- 8 M18 GALVANIZADA ...,...,"60,00","99,00","99,00","258,00","0,39","2,71",,,Q TRM/26,0.0
4055,Repuestos Electromecánicos recurrentes,2,Vehículo,10% Suspensiones de Cabinas,Repuesto Mecánico,TDB,39800,2-01-00286,10000079,TUERCA HEXAG. DIN 985-8 M10 ELECTROGALV. (C.EL...,...,"0,00","130,00","130,00","260,00","0,23","1,60",,,Q TRM/26,0.0
4056,Repuestos Electromecánicos recurrentes,2,Torres,Reacondicionamiento de Poleas Palpadoras,Repuesto Mecánico,TDB,39800,2-01-00835,10006287,"RES.C. A=1,00/DIA.7,0/LSC=20,0 EE= 7,5; RES.D...",...,"72,00","72,00","72,00","216,00","0,27","1,88",,,Q TRM/26,0.0
4057,Repuestos Electromecánicos recurrentes,2,Torres,Reacondicionamiento de Poleas Palpadoras,Repuesto Mecánico,TDB,39800,2-01-00840,10873673,"ARANDELA ISO 7089 M20 300HV, FLZNNC-720H-L, RE...",...,"12,00","12,00","12,00","36,00","0,65","4,52",,,Q TRM/26,0.0


In [ ]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(df_melted):
    # Performed 4 aggregations grouped on columns: 'linea', 'Tipo de trabajo'
    df_melted = df_melted.groupby(['linea', 'Tipo de trabajo']).agg(IDTDBoIDAlmacn_first=('ID TDB o ID Almacén', 'first'), IDCotizado_first=('ID Cotizado', 'first'), DescripcindelBien_first=('Descripción del Bien', 'first'), value_sum=('value', 'sum')).reset_index()
    return df_melted

df_melted_clean = clean_data(df_melted.copy())
df_melted_clean.head()

In [128]:
# Columnas de referencia general
columnas_info = [
    'Categoria', 'Proceso', 'Ubicación', 'Tipo de Bien', 'TDB/LOCAL',
    'Partida', 'ID TDB o ID Almacén', 'ID Cotizado', 'Descripción del Bien',
    'Unid.', 'P.U. $us.', 'P.U. Bs'
]

# Agrupar correctamente
df_grouped = (
    df_melted.groupby(['linea', 'Tipo de trabajo'], as_index=False)
      .agg(
          {
              'value': 'sum',  # Sumar value
              **{col: 'first' for col in columnas_info}  # Mantener la primera aparición de cada columna informativa
          }
      )
)

# Mostrar el resultado
df_grouped


,linea,Tipo de trabajo,value,Categoria,Proceso,Ubicación,Tipo de Bien,TDB/LOCAL,Partida,ID TDB o ID Almacén,ID Cotizado,Descripción del Bien,Unid.,P.U. $us.,P.U. Bs
0,Q LAM/24,10% Suspensiones de Cabinas,00072180072018258210,Repuestos Electromecánicos recurrentes,2,Vehículo,Repuesto Mecánico,TDB,39800,2-01-00342,10182756,CASQUILLO CON BORDE GGB2215 DU GLACIER,Pza,"13,23","92,08"
1,Q LAM/24,Cambio Correas,0000000,Repuestos Electromecánicos recurrentes,,Estaciones,Repuesto Mecánico,TDB,39800,2-01-00595,10217238,CORREA TRAP. XPC LW=2000 MM DIN 7753-1,Pza,"92,68","645,98"
2,Q LAM/24,Cambio de Neumáticos,10,Repuestos Electromecánicos recurrentes,2,Estaciones,Repuesto Mecánico,TDB,34300,2-01-00873,12008441,"NEUMÁTICO ""DOPPELMAYR-K""",Pza,"191,97","1.336,11"
3,Q LAM/24,Lubricación de rodetes de apertura y cierre,0,Repuestos Electromecánicos recurrentes,,Estaciones,Repuesto Mecánico,TDB,39800,2-01-00650,10863213,Cepillo de lubricador de duckosil,Pza,"35,57","247,92"
4,Q LAM/24,Manto. Pinza,0001002840000003100017228242042443440102019214...,Repuestos Electromecánicos recurrentes,2,Vehículo,Repuesto Mecánico,TDB,39800,2-01-00651,11359422,LENGÜETA DE PLÁST. A108-C TIPO 3/510/IBAF Ø52-...,Pza,"91,95","639,97"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,Q TRM/26,Reacondicionamiento de Poleas Palpadoras,000000000000000000,Repuestos Electromecánicos recurrentes,2,Torres,Repuesto Mecánico,TDB,39800,2-01-00013,10539289,GUARN. GOMA C 258X69/R30/CONCAV.P.POL.SENSOR (...,Pza,"124,41","865,89"
359,Q TRM/26,Reacondicionamiento y Cambio de poleas de esta...,0000000000,Repuestos Electromecánicos recurrentes,2,Estaciones,Repuesto Mecánico,TDB,39800,2-01-00011,10431740,GUARN.DE GOMA 501 485X100/R40,Pza,"401,33","2.793,26"
360,Q TRM/26,Reacondionamiento y cambio de Poleas de torre,00000000000000000,Repuestos Electromecánicos recurrentes,2,Torres,Repuesto Mecánico,TDB,39800,2-01-00011,10431740,GUARN.DE GOMA 501 485X100/R40,Pza,"401,33","2.793,26"
361,Q TRM/26,Sistema RPD,00000000000,Repuestos Electromecánicos recurrentes,2,Torres,Repuesto Eléctrico,TDB,39700,2-02-00151,11317920,RPD INTERRUPTOR DE VIGILANCIA DEL CABLE GS 5...,Pza,"1.233,52","8.585,30"


In [144]:
#df_resumen = df_melted.groupby(['linea', 'Tipo de trabajo'], as_index=False).agg({
df_resumen = df_melted.groupby(['linea', 'Tipo de trabajo'],as_index=False).agg({
    'ID TDB o ID Almacén': 'first',
    'ID Cotizado': 'first',
    'Descripción del Bien': 'first',
    'value': 'sum'
}).sort_values(by=['linea', 'Tipo de trabajo'])
# filtrar los valores donde en la columna 'value' sea diferente de 0
df_resumen = df_resumen[df_resumen['value'] != 0]
df_resumen

,linea,Tipo de trabajo,ID TDB o ID Almacén,ID Cotizado,Descripción del Bien,value
0,Q LAM/24,10% Suspensiones de Cabinas,2-01-00342,10182756,CASQUILLO CON BORDE GGB2215 DU GLACIER,585.0
2,Q LAM/24,Cambio de Neumáticos,2-01-00873,12008441,"NEUMÁTICO ""DOPPELMAYR-K""",10.0
4,Q LAM/24,Manto. Pinza,2-01-00651,11359422,LENGÜETA DE PLÁST. A108-C TIPO 3/510/IBAF Ø52-...,1447.0
6,Q LAM/24,Reacondicionamiento de Poleas Palpadoras,2-01-00013,10539289,GUARN. GOMA C 258X69/R30/CONCAV.P.POL.SENSOR (...,168.0
7,Q LAM/24,Reacondicionamiento y Cambio de poleas de esta...,2-01-00011,10431740,GUARN.DE GOMA 501 485X100/R40,176.0
...,...,...,...,...,...,...
325,Q LV/26,Reacondicionamiento de Poleas Palpadoras,2-01-00013,10539289,GUARN. GOMA C 258X69/R30/CONCAV.P.POL.SENSOR (...,326.0
326,Q LV/26,Reacondicionamiento y Cambio de poleas de esta...,2-01-00011,10431740,GUARN.DE GOMA 501 485X100/R40,49.0
327,Q LV/26,Reacondionamiento y cambio de Poleas de torre,2-01-00011,10431740,GUARN.DE GOMA 501 485X100/R40,482.0
328,Q LV/26,Sistema RPD,2-02-00151,11317920,RPD INTERRUPTOR DE VIGILANCIA DEL CABLE GS 5...,15.0


In [138]:
df_resumen.unstack()

ID TDB o ID Almacén                 \
Tipo de trabajo 10% Suspensiones de Cabinas  Cambio Correas   
linea                                                         
Q LAM/24                          2-01-00342     2-01-00595   
Q LAM/25                          2-01-00342     2-01-00595   
Q LAM/26                          2-01-00342     2-01-00595   
Q LAZ/24                          2-01-00342     2-01-00595   
Q LAZ/25                          2-01-00342     2-01-00595   
Q LAZ/26                          2-01-00342     2-01-00595   
Q LB/24                           2-01-00342     2-01-00595   
Q LB/25                           2-01-00342     2-01-00595   
Q LB/26                           2-01-00342     2-01-00595   
Q LCA/24                          2-01-00342     2-01-00595   
Q LCA/25                          2-01-00342     2-01-00595   
Q LCA/26                          2-01-00342     2-01-00595   
Q LCE/24                          2-01-00342     2-01-00595   
Q LCE/25                          2-01-00342     2-01-00595   
Q LCE/26                          2-01-00342     2-01-00595   
Q LMO/24                          2-01-00342     2-01-00595   
Q LMO/25                          2-01-00342     2-01-00595   
Q LMO/26                          2-01-00342     2-01-00595   
Q LN/24                           2-01-00342     2-01-00595   
Q LN/25                           2-01-00342     2-01-00595   
Q LN/26                           2-01-00342     2-01-00595   
Q LPL/24                          2-01-00342     2-01-00595   
Q LPL/25                          2-01-00342     2-01-00595   
Q LPL/26                          2-01-00342     2-01-00595   
Q LR/24                           2-01-00342     2-01-00595   
Q LR/25                           2-01-00342     2-01-00595   
Q LR/26                           2-01-00342     2-01-00595   
Q LV/24                           2-01-00342     2-01-00595   
Q LV/25                           2-01-00342     2-01-00595   
Q LV/26                           2-01-00342     2-01-00595   
Q TRM/24                          2-01-00342     2-01-00595   
Q TRM/25                          2-01-00342     2-01-00595   
Q TRM/26                          2-01-00342     2-01-00595   

                                      \
Tipo de trabajo Cambio de Neumáticos   
linea                                  
Q LAM/24                  2-01-00873   
Q LAM/25                  2-01-00873   
Q LAM/26                  2-01-00873   
Q LAZ/24                  2-01-00873   
Q LAZ/25                  2-01-00873   
Q LAZ/26                  2-01-00873   
Q LB/24                   2-01-00873   
Q LB/25                   2-01-00873   
Q LB/26                   2-01-00873   
Q LCA/24                  2-01-00873   
Q LCA/25                  2-01-00873   
Q LCA/26                  2-01-00873   
Q LCE/24                  2-01-00873   
Q LCE/25                  2-01-00873   
Q LCE/26                  2-01-00873   
Q LMO/24                  2-01-00873   
Q LMO/25                  2-01-00873   
Q LMO/26                  2-01-00873   
Q LN/24                   2-01-00873   
Q LN/25                   2-01-00873   
Q LN/26                   2-01-00873   
Q LPL/24                  2-01-00873   
Q LPL/25                  2-01-00873   
Q LPL/26                  2-01-00873   
Q LR/24                   2-01-00873   
Q LR/25                   2-01-00873   
Q LR/26                   2-01-00873   
Q LV/24                   2-01-00873   
Q LV/25                   2-01-00873   
Q LV/26                   2-01-00873   
Q TRM/24                  2-01-00873   
Q TRM/25                  2-01-00873   
Q TRM/26                  2-01-00873   

                                                                          \
Tipo de trabajo Lubricación de rodetes de apertura y cierre Manto. Pinza   
linea                                                                      
Q LAM/24                                         2-01-00650   2-01-00651   
Q LAM/25   

In [96]:
df_general = pd.concat([df_recurrentes, df_no_recurrentes, df_adicionales], ignore_index=True)

In [97]:
df_general

2,Categoria,Proceso,Ubicación,Tipo de trabajo,Tipo de Bien,TDB/LOCAL,Partida,ID TDB o ID Almacén,ID Cotizado,Descripción del Bien,...,Q LCE/26,Q LCA/26,Q LV/26,Q TRM/26,Q 2026,Q Total,P.U. $us.,P.U. Bs,Observaciones,Comentarios
0,Repuestos Electromecánicos recurrentes,2,Torres,Reacondionamiento y cambio de Poleas de torre,Repuesto Mecánico,TDB,39800,2-01-00011,10431740,GUARN.DE GOMA 501 485X100/R40,...,66,20,50,0,"877,00","2.461,00","401,33","2.793,26",,
1,Repuestos Electromecánicos recurrentes,2,Torres,Reacondionamiento y cambio de Poleas de torre,Repuesto Mecánico,TDB,39800,2-01-00012,10434107,GUARN.DE GOMA 420 422X 93/R34,...,76,10,65,0,"579,00","1.332,00","317,23","2.207,92",,
2,Repuestos Electromecánicos recurrentes,,Torres,Reacondionamiento y cambio de Poleas de torre,Rodamientos,LOCAL,39800,2-01-00077,10501989,RRB DIN 625/1- 6310-2RS1-C3GLE VT210 (90% +/-5...,...,132,50,100,0,"1.942,00","4.983,00","62,07","432,01",Se descontaron cantidades adquiridas 2024,
3,Repuestos Electromecánicos recurrentes,2,Estaciones,Reacondicionamiento y Cambio de poleas de esta...,Repuesto Mecánico,TDB,39800,2-01-00011,10431740,GUARN.DE GOMA 501 485X100/R40,...,60,15,0,0,"471,00","1.009,00","401,33","2.793,26",,
4,Repuestos Electromecánicos recurrentes,2,Estaciones,Reacondicionamiento y Cambio de poleas de esta...,Repuesto Mecánico,TDB,39800,2-01-00074,10461436,GUARN.DE GOMA 403 403X 93/R38,...,36,15,0,0,"495,00","1.158,00","275,00","1.914,00",,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
513,Repuestos Electromecánicos no recurrentes,2,Estación,Mantenimientos Menores,Repuesto Mecánico,TDB,39800,,10999021,RODILLO DE PLASTICO DIA 80/17 B=50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SEGUN COTIZACION tdb-ofe-pfg00314-24-REP-VPP_001,NaN
514,Repuestos Electromecánicos no recurrentes,2,Estación,Mantenimientos Menores,Repuesto Mecánico,TDB,39800,,11023220,DIENTE DE TEFLON,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SEGUN COTIZACION tdb-ofe-pfg00314-24-REP-VPP_001,NaN
515,Repuestos Electromecánicos no recurrentes,2,Estación,Mantenimientos Menores,Repuesto Mecánico,TDB,39800,,10000594,"PASADOR SUJ, DIN 1481 10X50 CON RECUBR. DACROMET",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PARA CASQUILLOS LINEA VERDE,NaN
516,Repuestos Electromecánicos no recurrentes,2,Estación,Mantenimientos Menores,Repuesto Mecánico,TDB,39800,,10439266,ARANDELA D45 E=5 GALVAN P. UNIDAD HIDR. POCLAIN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PARA CASQUILLOS LINEA VERDE,NaN


In [ ]:
df_ovh  = clean_data(import_df_sheets(sheet_url, "Rep Ovh Bal"))
sheet_url_ovh = "https://docs.google.com/spreadsheets/d/1vB80x2ES2_is57j0jE4vJXCnmGYynO9GlQB-yr8pbVc/edit?gid=326635939#gid=326635939"

df_ovh_2 = clean_data(import_df_sheets(sheet_url_ovh,"Hoja1").copy(),column=3,ini=4)
df_ovh_2

3,Nro,Categoria,Proceso,Ubicación,Tipo de trabajo,Tipo de Bien,TDB/LOCAL,Partida,ID TDB o ID Almacén,ID Cotizado,Descripción del Bien,Q 2024,Q 2025,CANT 2025,Unid.,Observaciones
1,1,Repuestos mantenimientos mayores para estacion...,2,Torres,OVH balancines TRM,Repuesto Mecánico,TDB,39800,,10431740,GUARN.DE GOMA 501 485X100/R40,993,,993,Pza,
2,2,Repuestos mantenimientos mayores para estacion...,2,Torres,OVH balancines TRM,Rodamientos,LOCAL,39800,,10501989,RRB DIN 625/1- 6310-2RS1-C3GLE VT210 (90% +/-5...,,1768,1768,Pza,Se reducen 170 rodamientos compra 2024
3,3,Repuestos mantenimientos mayores para estacion...,2,Torres,OVH balancines TRM,Repuesto Mecánico,TDB,39800,,10293344,"PERNO DIA.29,9 L 225 M24 PARA POLEA TIPO...",539,,539,Pza,
4,4,Repuestos mantenimientos mayores para estacion...,2,Torres,OVH balancines TRM,Repuesto Mecánico,TDB,39800,,10252061,TUERCA AUTOFIJ. INLOC - 8 M24 C.ELEM.DE FIJAC...,251,,251,Pza,Se reducen 150u de compra T&M
5,5,Repuestos mantenimientos mayores para estacion...,2,Torres,OVH balancines TRM,Repuesto Mecánico,TDB,39800,,10083926,CASQUILLO PM 6040 DS TIPO KW SIN AGUJ.DE ENG...,449,,449,Pza,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,62,Repuestos mantenimientos mayores para estacion...,2,Torres,OVH balancines TRM,Repuesto Mecánico,TDB,39800,,10087005,TORNILLO DIN7984- 8.8 M12X 30,248,,248,Pza,
63,63,Repuestos mantenimientos mayores para estacion...,2,Torres,OVH balancines TRM,Repuesto Mecánico,TDB,39800,,11545052,"EJE DIÁ.90H8 LN=541, P. APOYO P. CP 4P/4S TIPO...",9,,9,Pza,
64,64,Repuestos mantenimientos mayores para estacion...,2,Torres,OVH balancines TRM,Repuesto Mecánico,TDB,39800,,10488001,CASQUILLO PM 9080 DS TIPO KW SIN AGUJERO DE EN...,66,,66,Pza,
65,65,Repuestos mantenimientos mayores para estacion...,2,Torres,OVH balancines TRM,Repuesto Mecánico,TDB,39800,,10871605,ARANDELA M24,121,,121,Pza,


In [ ]:
# Convertir columnas numericas
columnas_numericas = ['Q 2024', 'Q 2025', 'CANT 2025']
for col in columnas_numericas:
    df_ovh_2 = convert_column_to_numeric(df_ovh_2, col)


In [ ]:
# Filter df_ovh to include only items whose 'ID Cotizado' is present in df_ovh_2
df_ovh_filtered = df_ovh[~df_ovh['ID Cotizado'].isin(df_ovh_2['ID Cotizado'].unique())]
df_ovh_filtered ['CANT 2025'] = df_ovh_filtered['Q 2024'] +df_ovh_filtered['Q 2025']
df_ovh = pd.concat([df_ovh_filtered,df_ovh_2], ignore_index=True)

/tmp/ipykernel_147923/3534759073.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ovh_filtered ['CANT 2025'] = df_ovh_filtered['Q 2024'] +df_ovh_filtered['Q 2025']


,Categoria,Proceso,Ubicación,Tipo de trabajo,Tipo de Bien,TDB/LOCAL,Partida,ID TDB o ID Almacén,ID Cotizado,Descripción del Bien,...,Bs. 2025,Bs. 2026,Bs. Total,Observaciones,Comentarios,Entrega 2024,Entrega 2025,Entrega 2026,CANT 2025,Nro
0,Repuestos mantenimientos mayores para estacion...,2,Torres,OVH balancines TRM,Repuesto Mecánico,TDB,39800,2-01-00487,10000338,"ANILLO RETEN. DIN 471 85X3,0 FOSFAT.",...,"0,00","0,00","618,05",,,,,,30.0,NaN
1,Repuestos mantenimientos mayores para estacion...,2,Torres,OVH balancines TRM,Repuesto Mecánico,TDB,39800,2-01-00128,10000343,ANILLO RETEN. DIN 471 100X3 FOSFAT.,...,"0,00","0,00","843,55",,,,,,30.0,NaN
2,Repuestos mantenimientos mayores para estacion...,2,Torres,OVH balancines TRM,Repuesto Mecánico,TDB,39800,2-01-00488,10000381,CIRCLIP INTER. DIN 472 52X2 FOSFAT.,...,"0,00","0,00","144,07",,,,,,30.0,NaN
3,Repuestos mantenimientos mayores para estacion...,2,Torres,OVH balancines TRM,Repuesto Mecánico,TDB,39800,2-01-00035,10000400,"CIRCLIP INTER. DIN 472 80X2,5 FOSFAT.",...,"0,00","0,00","336,17",,,,,,30.0,NaN
4,Repuestos mantenimientos mayores para estacion...,2,Torres,OVH balancines TRM,Repuesto Mecánico,TDB,39800,2-01-00036,10000409,CIRCLIP INTER. DIN 472 110X4 FOSFAT.,...,"0,00","0,00","987,62",,,,,,30.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,Repuestos mantenimientos mayores para estacion...,2,Torres,OVH balancines TRM,Repuesto Mecánico,TDB,39800,,10087005,TORNILLO DIN7984- 8.8 M12X 30,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,248,62
105,Repuestos mantenimientos mayores para estacion...,2,Torres,OVH balancines TRM,Repuesto Mecánico,TDB,39800,,11545052,"EJE DIÁ.90H8 LN=541, P. APOYO P. CP 4P/4S TIPO...",...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,9,63
106,Repuestos mantenimientos mayores para estacion...,2,Torres,OVH balancines TRM,Repuesto Mecánico,TDB,39800,,10488001,CASQUILLO PM 9080 DS TIPO KW SIN AGUJERO DE EN...,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,66,64
107,Repuestos mantenimientos mayores para estacion...,2,Torres,OVH balancines TRM,Repuesto Mecánico,TDB,39800,,10871605,ARANDELA M24,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,121,65
